In [4]:
import pandas as pd

pd.set_option('display.max_columns',None)

In [5]:
df_ksxx = pd.read_csv('./data/test_ksxx.csv')
df_ksxx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43400 entries, 0 to 43399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   考生号     43400 non-null  int64 
 1   性别      43400 non-null  object
 2   县区代码    43400 non-null  int64 
 3   学校代码    43400 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.3+ MB


In [6]:
df_ksxx.head()

,考生号,性别,县区代码,学校代码
0,60898139,女,9076,593115
1,42872499,女,160,474412
2,73504698,男,9076,593115
3,98149243,男,160,474412
4,91911749,女,160,474412


In [7]:
df_kscj = pd.read_csv('./data/test_kscj.csv')
df_kscj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43404 entries, 0 to 43403
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   考生号     43404 non-null  int64  
 1   语文      43404 non-null  int64  
 2   数学      43404 non-null  int64  
 3   英语      43404 non-null  int64  
 4   物理      43404 non-null  int64  
 5   化学      43404 non-null  int64  
 6   生物      43404 non-null  int64  
 7   政治      43404 non-null  int64  
 8   历史      43404 non-null  int64  
 9   地理      43404 non-null  int64  
 10  体育      43404 non-null  float64
dtypes: float64(1), int64(10)
memory usage: 3.6 MB


In [8]:
df_kscj.head()

,考生号,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育
0,38096712,96,72,61,82,59,62,56,62,48,35.6
1,23257273,103,74,118,74,65,38,47,48,48,29.4
2,95989074,119,117,114,93,83,87,77,88,89,38.0
3,36935864,102,107,112,82,78,77,77,78,79,38.7
4,78065505,120,125,137,87,81,84,90,95,93,35.2


In [9]:
df_ksxx = df_ksxx.drop_duplicates(subset=['考生号'],keep='first')
df_ksxx.shape[0]

43395

In [10]:
df_kscj = df_kscj.drop_duplicates(subset=['考生号'],keep='first')
df_kscj.shape[0]

43399

In [11]:
df2 = pd.merge(df_ksxx,df_kscj,how='left', on='考生号')
df2.head()

,考生号,性别,县区代码,学校代码,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育
0,60898139,女,9076,593115,72,43,48,55,41,32,31,61,44,28.9
1,42872499,女,160,474412,113,114,132,86,77,88,87,90,86,38.9
2,73504698,男,9076,593115,14,55,39,35,0,0,12,79,48,36.4
3,98149243,男,160,474412,100,112,61,81,67,85,79,85,91,39.5
4,91911749,女,160,474412,103,109,122,83,54,66,71,75,72,38.4


In [12]:
df2.fillna(0, inplace=True)
df2.shape[0]

43395

In [13]:
df2 = df2[df2['考生号'] != 0]
df2.shape[0]

43395

In [29]:
# school_counts = df2.groupby('学校代码')['县区代码'].nunique()
# max_count_school = school_counts.idxmax()
# max_count_district = df2.loc[df2['学校代码'] == max_count_school, '县区代码'].mode()[0]
# df2.loc[df2['学校代码'] == max_count_school, '县区代码'] = max_count_district
# df2.info()

# def merge_schools(schools):
#     school_count = {}
#     for school, district in schools:
#         if school in school_count:
#             school_count[school][district] = school_count[school].get(district, 0) + 1
#         else:
#             school_count[school] = {district: 1}
    
#     merged_schools = {}
#     for school, districts in school_count.items():
#         max_district = max(districts, key=districts.get)
#         merged_schools[school] = max_district
    
#     return merged_schools

In [30]:
df2['总分']  = df2['语文'] + df2['数学'] + df2['英语'] + df2['物理'] * 0.6 + df2['化学'] * 0.6 + df2['生物'] * 0.3 + df2['政治'] * 0.5 + df2['历史'] * 0.5 +df2['地理'] * 0.3 +df2['体育'] 
df2.head()

,考生号,性别,县区代码,学校代码,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育,总分
0,60898139,女,9076,593115,72,43,48,55,41,32,31,61,44,28.9,318.3
1,42872499,女,160,474412,113,114,132,86,77,88,87,90,86,38.9,636.4
2,73504698,男,9076,593115,14,55,39,35,0,0,12,79,48,36.4,225.3
3,98149243,男,160,474412,100,112,61,81,67,85,79,85,91,39.5,536.1
4,91911749,女,160,474412,103,109,122,83,54,66,71,75,72,38.4,569.0


In [76]:
# 1.2 部分学校出现在多个县区中，请将学校保留在考生最多的县区中，并将其它县区改为这个县区。
# groupby两次，先groupby学校，再遍历list，对每个学校groupby县区，在此list中取最值，将最大值在同个县区的学校信息录入同一工作簿

school_list = df2.groupby('学校代码')

result_df = pd.DataFrame(columns=['学校代码','最多出现的县区代码'])
# 遍历每个学校
for school_code, group_df in school_list:
    # 对每个学校再按照“县区代码”进行分组，并计算每个县区代码的出现次数
    county_counts = group_df['县区代码'].value_counts()
    # 找到出现次数最多的县区代码
    most_common_county = county_counts.idxmax()
    # 将结果添加到结果 DataFrame 中
    result_df = result_df.append({'学校代码': school_code, '最多出现的县区代码': most_common_county}, ignore_index=True)

# 将两个 DataFrame 进行连接（join），连接条件是“学校代码”
merged_df = pd.merge(df2, result_df, on='学校代码', how='left')

# 将“最多出现的县区代码”写入到 df2 中
merged_df['县区代码'] = merged_df['最多出现的县区代码']

# 删除辅助列
merged_df.drop(columns=['最多出现的县区代码'], inplace=True)

# 创建一个 ExcelWriter 对象
with pd.ExcelWriter('output.xlsx') as writer:
    # 遍历每个县区代码
    for county_code in merged_df['县区代码'].unique():
        # 获取具有相同县区代码的数据
        county_data = merged_df[merged_df['县区代码'] == county_code]
        # 将数据写入到对应的工作簿中，工作簿名称为县区代码
        county_data.to_excel(writer, sheet_name=str(county_code), index=False)

In [58]:
# for xqdm,df_xq in xq_group:
#     print(xqdm)
#     print(df_xq.head())


In [33]:
import pandas as pd

# 计算并格式化单一数值
# 创建一个空的DataFrame来存储统计数据
df_statistics = pd.DataFrame(columns=[
    '县区代码', '考生数', '与考数', '与考率', '语数英三科均分率',
    '十科均分率', '优秀率', '单科及格率', '四率综合比', '排名'
])

# 假设 xq_group 是已按县区代码分组的 DataFrameGroupBy 对象
for xqdm, df_xq in xq_group:
    # 计算考生数
    total_students = df_xq.shape[0]

    # 计算与考数（没有任何科目为0分的视为与考）
    df_with_scores = df_xq[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].gt(0).all(axis=1)
    num_attended = df_with_scores.sum()

    # 计算与考率
    attendance_rate = (num_attended / total_students) * 100
    
    # 计算语数英三科均分率
    df_yuyingmath_scores = df_xq[['语文', '数学', '英语']].sum(axis=1)
    yuyingmath_average = df_yuyingmath_scores.mean() / 450 * 100  # 确保平均值是单个数值
    
    # 计算十科均分率
    df_all_scores = df_xq[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].sum(axis=1)
    all_average = df_all_scores.mean() / 800 * 100  # 确保得到平均值
    
    # 计算优秀率（总分大于等于 640 为优秀）
    excellent_students = df_xq[df_xq[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].sum(axis=1) >= 640]
    excellent_rate = (excellent_students.shape[0] / total_students) * 100
    
    # 计算单科及格率
    pass_rates = []
    for subject in ['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']:
        passing_count = df_xq[df_xq[subject] >= 90].shape[0]
        pass_rate = passing_count / total_students
        pass_rates.append(pass_rate * 100)  # 确保是百分比形式
    
    single_subject_pass_rate = sum(pass_rates)  # 转化为百分比之和
    
    # 计算四率综合比
    overall_rate = ((yuyingmath_average + all_average) * 0.3 + (excellent_rate + single_subject_pass_rate) * 0.2)
    
    # 将统计结果添加到 DataFrame
    df_statistics = df_statistics.append({
        '县区代码': xqdm,
        '考生数': total_students,
        '与考数': num_attended,
        '与考率': f"{attendance_rate:.2f}%",
        '语数英三科均分率': f"{yuyingmath_average:.2f}%",
        '十科均分率': f"{all_average:.2f}%",
        '优秀率': f"{excellent_rate:.2f}%",
        '单科及格率': f"{single_subject_pass_rate:.2f}%",
        '四率综合比': f"{overall_rate:.2f}%",
        '排名': None
    }, ignore_index=True)

# 按照四率综合比进行排序，并更新排名
df_statistics = df_statistics.sort_values(by='四率综合比', ascending=False)
df_statistics['排名'] = list(range(1, len(df_statistics) + 1))

# 保存到 Excel 文件
df_statistics.to_excel('zlfx.xlsx', sheet_name='县区', index=False)

In [86]:
import pandas as pd 
xq_school = pd.read_excel('output.xlsx')
# print(xq_school)
xq_school_group = df2.groupby(['县区代码', '学校代码'])

In [87]:
df_statistics = pd.DataFrame(columns=[
    '县区代码', '学校代码', '考生数', '与考数', '与考率', '语数英三科均分率',
    '十科均分率', '优秀率', '单科及格率', '四率综合比', '排名'
])

# 假设 xq_group 是已按县区代码分组的 DataFrameGroupBy 对象
for xqdm, df_xq in xq_group:
    # 假设 sch_group 是已按学校代码分组的 DataFrameGroupBy 对象
    for sch_code, df_sch in df_xq.groupby('学校代码'):
        # 计算考生数
        total_students = df_sch.shape[0]

        # 计算与考数（没有任何科目为0分的视为与考）
        df_with_scores = df_sch[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].gt(0).all(axis=1)
        num_attended = df_with_scores.sum()

        # 计算与考率
        attendance_rate = (num_attended / total_students) * 100

        # 计算语数英三科均分率
        df_yuyingmath_scores = df_sch[['语文', '数学', '英语']].sum(axis=1)
        yuyingmath_average = df_yuyingmath_scores.mean() / 450 * 100  # 确保平均值是单个数值

        # 计算十科均分率
        df_all_scores = df_sch[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].sum(axis=1)
        all_average = df_all_scores.mean() / 800 * 100  # 确保得到平均值

        # 计算优秀率（总分大于等于 640 为优秀）
        excellent_students = df_sch[df_sch[['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']].sum(axis=1) >= 640]
        excellent_rate = (excellent_students.shape[0] / total_students) * 100

        # 计算单科及格率
        pass_rates = []
        for subject in ['语文', '数学', '英语', '物理', '化学', '生物', '地理', '政治', '历史', '体育']:
            passing_count = df_sch[df_sch[subject] >= 90].shape[0]
            pass_rate = passing_count / (total_students * 10)
            pass_rates.append(pass_rate * 100)  # 确保是百分比形式

        single_subject_pass_rate = sum(pass_rates)  # 转化为百分比之和

        # 计算四率综合比
        overall_rate = ((yuyingmath_average + all_average) * 0.3 + (excellent_rate + single_subject_pass_rate) * 0.2)

        # 将统计结果添加到 DataFrame
        df_statistics = df_statistics.append({
            '县区代码': xqdm,
            '学校代码': sch_code,
            '考生数': total_students,
            '与考数': num_attended,
            '与考率': f"{attendance_rate:.2f}%",
            '语数英三科均分率': f"{yuyingmath_average:.2f}%",
            '十科均分率': f"{all_average:.2f}%",
            '优秀率': f"{excellent_rate:.2f}%",
            '单科及格率': f"{single_subject_pass_rate:.2f}%",
            '四率综合比': f"{overall_rate:.2f}%",
            '排名': None
        }, ignore_index=True)

# 按照四率综合比进行排序，并更新排名
df_statistics = df_statistics.sort_values(by='四率综合比', ascending=False)
df_statistics['排名'] = list(range(1, len(df_statistics) + 1))

# 保存到 Excel 文件
df_statistics.to_excel('zlfx1.xlsx', sheet_name='县区', index=False)